In [40]:
import keras
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from nets import inception_v3
from datasets import dataset_utils
from datasets import imagenet
from preprocessing import inception_preprocessing
from tensorflow.contrib import slim
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data

In [12]:
url = 'http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz'
checkpoints_dir = 'C:\\Users\\serge\\checkpoints'
dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

>> Downloading inception_v3_2016_08_28.tar.gz 100.0%
Successfully downloaded inception_v3_2016_08_28.tar.gz 100885009 bytes.


In [23]:
# InceptionV3 = keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
# InceptionV3.predict(elephant)

# Описание модели с использованием свёрточной части inception v3 

In [55]:
tf.reset_default_graph()

image_size = inception_v3.inception_v3.default_image_size
image=tf.placeholder(dtype='float32',shape=[28,28,3])

img_processed = inception_preprocessing.preprocess_image(image, image_size, image_size, is_training=False)
processed_images  = tf.expand_dims(img_processed, 0)

with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
    logits, tensors_list = inception_v3.inception_v3(processed_images, num_classes=1001, is_training=False)

llayer = tensors_list.get('PreLogits',None)
init_fn = slim.assign_from_checkpoint_fn(
    os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
    slim.get_model_variables())

Tensor("eval_image/Mul:0", shape=(299, 299, 3), dtype=float32)


# Функция возвращающая данные для 2-х указанных классов из датасета MNIST

In [48]:
def load_mnist_dataset(binary, label1, label2):
    mnist = input_data.read_data_sets("MNIST_data/", reshape=True, one_hot=False)
 
    X_train = mnist.train.images
    target_train = mnist.train.labels
    X_test = mnist.test.images
    target_test = mnist.test.labels

    if (binary):
        # Get only the samples with zero and one label for training.
        index_list_train = []
        for sample_index in range(target_train.shape[0]):
            label = target_train[sample_index]
            #if label == 1 or label == 0:
            if (label == label1 or label == label2):
                index_list_train.append(sample_index)

        # Reform the train data structure.
        X_train = mnist.train.images[index_list_train]
        target_train = mnist.train.labels[index_list_train]

        # Get only the samples with four and nine label for test set.
        index_list_test = []
        for sample_index in range(target_test.shape[0]):
            label = target_test[sample_index]
            #if label == 1 or label == 0:
            if (label == label1 or label == label2):
                index_list_test.append(sample_index)    

        # Reform the test data structure.
        X_test = mnist.test.images[index_list_test]
        target_test = mnist.test.labels[index_list_test]

        target_train[target_train == label1] = 0;
        target_train[target_train == label2] = 1;
        
        target_test[target_test == label1] = 0;
        target_test[target_test == label2] = 1;
        
    return (X_train, X_test, target_train, target_test)

In [100]:
(X_tr, X_te, target_train, target_test) = load_mnist_dataset(binary = True, label1 = 4, label2 = 9)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [101]:
def triple (X):
    new_X = []
    for img in X:
        s_im =  np.stack((img, )*3, -1)
        new_X.append(s_im)
    return np.array(new_X).reshape(len(X), 28, 28, 3)

In [103]:
X_test = triple(X_te)
X_train = triple(X_tr)
print(X_train.shape, X_test.shape)

(10761, 28, 28, 3) (1991, 28, 28, 3)


# Подготовка признаков с использованием Inceptionv3

In [104]:
X_dims = np.array([None]*X_train.shape[0])

n_train = X_train.shape[0]
n_test = X_test.shape[0]

#X_train = X_train.reshape((784,1))
#target_test = target_test.reshape((n_test,1))

n_epoch = 35
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    init_fn(sess)
    for i in range(len(X_train)):
        last = sess.run([llayer],feed_dict={image:X_train[i,:]})
        X_dims[i]=np.squeeze(last)

INFO:tensorflow:Restoring parameters from C:\Users\serge\checkpoints\inception_v3.ckpt
[0.6819549  0.06001298 0.2397255  ... 0.23592508 0.22953045 0.8219075 ]
[1.0460186  0.294665   0.35787088 ... 0.23880139 0.42690063 0.9695197 ]
[0.82600594 0.23302957 0.07579592 ... 0.09121108 0.19748732 1.118968  ]
[0.6643816  0.23935267 0.6645012  ... 0.16762927 0.0588962  0.1738266 ]
[0.7728529  0.04904056 0.20339152 ... 0.8219329  0.10969191 0.5394596 ]
[0.68675166 0.33810598 0.21807906 ... 0.586233   0.52102417 0.5129867 ]
[0.6895923  0.09158283 0.20006706 ... 0.41457647 0.52074844 0.5648154 ]
[0.6996802  0.20300382 0.07333773 ... 0.40675592 0.12795827 0.5236784 ]
[0.86165667 0.33278325 0.22392748 ... 0.26013815 0.6344536  0.62280035]
[0.8261528  0.26088732 0.19662806 ... 0.6953953  0.5089308  0.88043064]
[0.833093   0.34596    0.1290303  ... 0.8564201  0.6584602  0.96600544]
[0.7930305  0.14848278 0.17179638 ... 0.7293493  0.17905445 0.9711609 ]
[1.2022917  0.3533922  0.27906364 ... 0.25905415 

[0.9009171  0.03351505 0.28482243 ... 0.21871419 0.28497893 0.6082784 ]
[0.8161478  0.42716298 0.2588443  ... 0.47339505 0.6597441  0.7163933 ]
[0.23071727 0.25028703 0.32710162 ... 0.21847656 0.2124755  0.27698213]
[0.8862666  0.17667829 0.29538578 ... 0.23161891 0.3718956  0.6854259 ]
[0.7436852  0.15965798 0.3647491  ... 0.24101156 0.5687165  0.70535576]
[0.49887457 0.24986397 0.19012174 ... 0.42825472 0.07971488 0.4445712 ]
[0.5488312  0.07349093 0.12930013 ... 0.5520938  0.17056274 0.15127485]
[0.36684188 0.6584499  0.25619954 ... 1.1333735  0.59836555 0.64597774]
[0.46812743 1.004868   0.29717857 ... 1.1604835  0.63805306 0.44402996]
[0.42383298 0.6796752  0.11194045 ... 0.9601147  0.4164805  1.0238526 ]
[0.31238806 0.5721889  0.21449389 ... 0.5337849  0.73581606 0.65560263]
[0.87725616 0.10653938 0.44533682 ... 0.4197646  0.02917403 0.74346626]
[1.0037347  0.330024   0.35200986 ... 0.26919812 0.49240708 1.4380841 ]
[0.58704615 0.06498787 0.61184514 ... 0.7564318  0.3022329  0.97

[0.66218966 0.27112722 0.60092276 ... 0.12809321 0.46119753 0.3386335 ]
[0.5601614  0.03032325 0.17706008 ... 0.8471514  0.22441742 0.25569126]
[0.36468735 0.12970605 0.23964025 ... 0.21281601 0.32822943 0.43101543]
[0.23599954 0.10695605 0.22723521 ... 0.56846833 0.15413913 0.0211542 ]
[0.91402835 0.43210858 0.0489693  ... 0.4666618  0.85087484 1.0798675 ]
[0.821615   0.07520029 0.17827411 ... 0.8247934  0.64959586 0.6198517 ]
[0.6630304  0.08089064 0.06785768 ... 0.55374384 0.58224696 0.6681053 ]
[0.90100986 0.11787733 0.21891965 ... 0.6884471  0.33926418 0.78395444]
[0.58748883 0.34623903 0.22134817 ... 0.83816904 0.56196606 0.77542734]
[0.67659855 0.20993195 0.29745948 ... 0.5889376  0.17820896 0.80751497]
[0.33441138 0.4332147  0.08343151 ... 1.0887771  0.45670974 0.822935  ]
[0.6719383  0.18164182 0.4803065  ... 0.6149004  0.14360017 0.6748004 ]
[1.0969213  0.10437294 0.26375815 ... 0.3140518  0.04496647 0.7506244 ]
[0.72128993 0.07287352 0.32852235 ... 0.5500246  0.1101642  0.88

[0.9998824  0.1296239  0.8137428  ... 0.2729015  0.41081682 0.84175354]
[1.080565   0.18959107 0.64926606 ... 0.20221946 0.34599543 1.2048602 ]
[0.6271719  0.0584683  0.21547042 ... 0.3787441  0.21027924 1.1080322 ]
[1.0176342  0.15257156 0.10708961 ... 0.67301035 0.7528742  1.0038399 ]
[0.44180954 0.4234625  0.17867975 ... 0.5303138  0.3642584  0.8002796 ]
[0.87474036 0.01616356 0.27032334 ... 0.42170724 0.12466785 0.48137504]
[0.60865927 0.43034297 0.13393575 ... 0.6799609  0.23766927 0.62573326]
[0.4944054 0.1378855 0.1578452 ... 0.5963117 0.660371  0.7389253]
[0.7202994  0.08117498 0.16040626 ... 0.34350458 0.39782873 0.93911254]
[0.758004   0.03811964 0.7262654  ... 0.177039   0.5910887  0.76048326]
[0.57881016 0.08438502 0.22746581 ... 0.2969609  0.11360873 0.9669883 ]
[0.81829506 0.20874664 0.42859778 ... 0.10568748 0.66639805 0.60660625]
[1.0903319  0.21168555 0.5684115  ... 0.36885196 0.30335385 0.79177254]
[0.85893005 0.26178992 0.12808527 ... 0.3726718  0.2840742  0.6279971 

[0.7322567  0.3476086  0.75835174 ... 0.305141   0.8239704  1.0324428 ]
[0.2811049  0.04967369 0.12477618 ... 0.8030899  0.19308743 0.5474687 ]
[0.91165406 0.30213356 0.6664667  ... 0.4982406  0.63428503 0.99694556]
[0.10237799 0.54602206 0.28602087 ... 0.67826873 0.63780487 0.7997632 ]
[0.78605896 0.2199082  0.48591456 ... 0.62443155 0.27446732 0.36577982]
[0.6951049  0.16770065 0.57968104 ... 0.39741805 0.26825982 0.16860127]
[0.70544505 0.2947267  0.33320597 ... 0.3270985  0.52522206 0.6271168 ]
[0.1516978  0.41324693 0.42901248 ... 0.87743497 0.11740793 0.48243955]
[0.6761489  0.33612126 0.25818774 ... 0.44288465 0.25152653 0.583176  ]
[0.5332826  0.42893434 0.17708828 ... 0.538689   0.63833314 0.660562  ]
[0.4233908  0.34066892 0.5551407  ... 0.39043757 0.24883671 0.18692861]
[0.44872215 0.4071094  0.3873661  ... 0.83205056 0.4310223  0.5016066 ]
[0.59276366 0.37416357 0.20844838 ... 0.43438548 0.04163308 0.80251336]
[0.8337007  0.02212702 0.13772959 ... 0.16858128 0.06347096 1.11

...

[0.5502522  0.07581217 0.2958719  ... 0.31910005 0.08745454 0.49312854]
[0.7639412  0.1399466  0.29524553 ... 0.55350316 0.29389927 0.7778551 ]
[0.6172844  0.21101777 0.16301316 ... 0.39695916 0.32840797 0.93021494]
[0.88830143 0.1403832  0.26749974 ... 0.31805167 0.221225   0.42991582]
[0.4600705  0.30241224 0.19000976 ... 0.37925348 0.44587994 0.94615704]
[0.7074809  0.13693435 0.64205927 ... 0.23952337 0.         0.36587328]
[0.9416569  0.01534711 0.28648356 ... 0.32844773 0.21441741 0.3411233 ]
[0.74182326 0.13630544 0.3368932  ... 0.6224925  0.16306055 0.96997696]
[0.83652115 0.07942231 0.14245881 ... 0.19708583 0.14675824 0.5856471 ]
[0.84963506 0.2004244  0.5855468  ... 0.36653575 0.5801807  0.7088204 ]
[1.1517065  0.02631377 0.49382934 ... 0.3215108  0.02274977 0.69836485]
[0.7285755  0.23309208 0.17264609 ... 0.6773089  0.08261719 0.7317265 ]
[0.39080387 0.15536821 0.28474626 ... 0.6965144  0.06179237 0.53925747]
[0.6076268  0.510013   0.5022786  ... 0.30423072 0.54234266 0.50

[0.5766369  0.21824439 0.2292548  ... 0.60406244 0.2863571  0.9093191 ]
[0.674248   0.06605984 0.4350044  ... 0.49600735 0.01399065 0.14820604]
[0.5915417  0.34389132 0.45796543 ... 0.7821149  0.14620523 0.81621635]
[0.5477727  0.1798662  0.6488238  ... 0.12523961 0.4108711  0.28345388]
[0.7749602  0.28605598 0.3220026  ... 0.4770061  0.07214946 0.51976484]
[0.59592986 0.06777327 0.19309345 ... 0.40253365 0.22526516 0.3842439 ]
[0.39393315 0.31497887 0.11122047 ... 0.70768756 0.67978597 0.7756914 ]
[0.6972027  0.2207059  0.5371638  ... 0.32131934 0.07519437 0.43343496]
[0.4573282  0.47233632 0.38816857 ... 0.24134855 0.2384902  0.40322834]
[0.6130791  0.24033666 0.3117143  ... 0.49623457 0.12366925 0.22362635]
[0.59766763 0.4589795  0.41492906 ... 0.22974074 0.3136437  0.3798218 ]
[0.58576125 0.13478316 0.5642312  ... 0.0773694  0.05126569 0.29067636]
[0.60423476 0.30367023 0.40673438 ... 0.44555628 0.51326525 0.2563447 ]
[0.7109471  0.7715217  0.2473366  ... 0.55503875 0.41210106 0.32

[0.8794298  0.21328145 0.7163533  ... 0.37263834 0.31285366 1.2793593 ]
[0.35396472 0.44241157 0.08674169 ... 0.7042192  0.2543424  0.6777573 ]
[0.9393052  0.33658075 0.6668003  ... 0.43739316 0.         0.57739884]
[0.7644044  0.41353324 0.23533167 ... 0.65512425 0.21659757 0.6636791 ]
[0.30061308 0.302731   0.26191255 ... 0.9103134  0.50448525 0.64249927]
[0.31807345 0.6595818  0.24973701 ... 1.1198463  0.3558438  0.49767587]
[0.5433176  0.70022213 0.28008872 ... 0.6551003  0.5070353  0.34721363]
[0.8403193  0.16796751 0.32429034 ... 0.2874471  0.302442   0.89392674]
[0.93830484 0.11076299 0.6583423  ... 0.14248486 0.08039307 0.49323234]
[0.6346772  0.21232733 0.38920143 ... 0.02274093 0.13434757 0.37452704]
[0.35000053 0.05847159 0.20430574 ... 0.6710726  0.2654171  0.48996317]
[0.1736963  0.36537054 0.24141045 ... 0.88562757 0.4027106  0.4743372 ]
[0.6807261  0.34618258 0.7271891  ... 0.26039848 0.05394402 0.6331209 ]
[0.3089385  0.25411314 0.11734047 ... 0.9966051  0.18160504 0.29

[0.7696898  0.2997069  0.16584027 ... 0.16051595 0.06809218 0.6441817 ]
[0.7803985  0.3748069  0.20902196 ... 0.5398706  0.00284177 0.21954475]
[0.5176269  0.27367553 0.1710893  ... 0.34601107 0.13317662 0.9002279 ]
[0.7727056  0.05689877 0.24599665 ... 0.10197152 0.08323236 0.14171897]
[0.8667081  0.3349063  0.3294799  ... 0.29424554 0.0067773  0.31474185]
[0.6340237  0.5415229  0.37640604 ... 0.4951687  0.3794518  0.24743044]
[0.8426023  0.42759362 0.25342825 ... 0.3533512  0.2250419  0.91453254]
[0.6771713  0.07337322 0.26578632 ... 0.52982    0.45117337 0.48434854]
[0.7371871  0.32472378 0.3107855  ... 0.48022503 0.04859127 0.8200542 ]
[0.8850461  0.5932132  0.03902151 ... 0.54933584 0.52572584 0.82051694]
[0.8050069  0.5000403  0.23492287 ... 0.46550778 0.40929973 0.38847426]
[0.6321444  0.1598278  0.19185741 ... 0.6333023  0.48807696 0.7248299 ]
[0.6954686  0.4361729  0.08049949 ... 0.42441347 0.11615065 0.79137963]
[0.521725   0.35246146 0.39711758 ... 0.4758115  0.27742064 0.60

[0.97308475 0.15527423 0.4119686  ... 0.28460357 0.06903173 0.75568426]
[0.4435858  0.7544345  0.21836753 ... 0.6991391  0.265411   0.30322152]
[0.7138223  0.46429256 0.45612553 ... 0.39212242 0.23785423 0.52522624]
[0.77087724 0.1925896  0.07342985 ... 0.2925105  0.22957523 0.49000573]
[0.6479949  0.3294393  0.6767903  ... 0.390519   0.20037496 0.47427946]
[0.406249   0.2822173  0.05770265 ... 0.5226719  0.06014612 0.29933074]
[0.38746065 0.23564276 0.5166729  ... 0.87794524 0.16041584 0.16595198]
[0.95906144 0.07759802 0.06345462 ... 0.50100195 0.5927557  0.2820868 ]
[0.81547385 0.11814678 0.25248966 ... 0.10240141 0.         0.14506951]
[0.8993844  0.13599327 0.1947027  ... 0.09664553 0.40019923 0.869108  ]
[0.60106915 0.7505937  0.11667476 ... 0.52595633 0.31318018 0.34736097]
[0.57877684 0.09258718 0.3905185  ... 0.2347645  0.33976227 0.8376037 ]
[0.40634406 0.02566754 0.06435121 ... 0.5207268  0.2391187  1.0379822 ]
[0.35380486 0.5673043  0.08643443 ... 0.56653464 0.10346893 0.19

In [105]:
for i in range(X_dims.shape[0]):
    X_dims[i] = np.array(X_dims[i])
X_dims_new = np.vstack([x for x in X_dims])

# Кластеризация при помощи K-means

In [108]:
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=2,n_init=20)
clustering.fit(X_dims_new)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=20, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [119]:
pred = clustering.labels_
accuracy = np.mean(np.equal(pred,target_train))
print(accuracy)

0.9114394572995075


# Кластеризация при помощи алгоритма Agglomerative

In [123]:
from sklearn.cluster import AgglomerativeClustering

In [126]:
clustering1 = AgglomerativeClustering(n_clusters=2)
clustering1.fit(X_dims_new)

0.059381098410928354


In [127]:
pred1 = 1 - clustering1.labels_
accuracy = np.mean(np.equal(pred1,target_train))
print(accuracy)

0.9406189015890717
